In [1]:
import torch
import torch.nn as nn
import torchvision #containing: datasets, models, transforms, utils
import torchvision.transforms as transforms


# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [2]:
"""
torchvision.datasets: containing
    cifar
    coco
    fakedata
    folder
    lsun
    mnist
    omniglot
    phototour
    semeion
    stl10
    svhn
    utils
"""
# MNIST dataset (images and labels)
"""
torchvision.datasets.MNIST:
root (string): Root directory of dataset where ``processed/training.pt``
           and  ``processed/test.pt`` exist.
train (bool, optional): If True, creates dataset from ``training.pt``,
           otherwise from ``test.pt``.
download (bool, optional): If true, downloads the dataset from the internet and
           puts it in root directory. If dataset is already downloaded, it is not
           downloaded again.
transform (callable, optional): A function/transform that  takes in an PIL image
           and returns a transformed version. E.g, ``transforms.RandomCrop``
target_transform (callable, optional): A function/transform that takes in the
           target and transforms it.
"""
"""
train_dataset.train_data.shape: torch.Size([60000, 28, 28])
train_dataset.train_labels.shape: torch.Size([60000])
"""
# in this example, train_dataset.train_data, train_dataset.train_labels are still tensor even if transofrm is disabled
# but when iterating train_loader, it throws TypeError: batch must contain tensors, numbers, dicts or lists; found <class 'PIL.Image.Image'>
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor()
                                         )

print('data count:', len(train_dataset.train_data))

data count: 60000


In [3]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [4]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [5]:
"""torch.nn.modules.loss.CrossEntropyLoss
 `input` has to be a Tensor of size either :math:`(minibatch, C)` or
 |  :math:`(minibatch, C, d_1, d_2, ..., d_K)`
 |  with :math:`K \geq 2` for the `K`-dimensional case (described later).
 |  
 |  This criterion expects a class index (0 to `C-1`) as the
 |  `target` for each value of a 1D tensor of size `minibatch`
Note: input should be size: [batch_size, num_class]
Note: target should be size: [batch_size]
Note: target should be type torch.LongTensor. (of type torch.IntTensor will not work)
"""

#input: logits for different class
#target: the correct class id
criterion(torch.Tensor([[0.3,0.7]]), torch.Tensor([0]).type(torch.LongTensor))

tensor(0.9130)

In [6]:
# Train the model
total_step = len(train_loader) #equals to len(train_dataset.train_data)/batch_size=60000/100=600
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # Forward pass
        outputs = model(images)
        # outputs.shape: torch.Size([100, 10]), labels.shape: torch.Size([100])
        # outputs.type(): torch.FloatTensor, lables.type(): torch.LongTensor
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.1669
Epoch [1/5], Step [200/600], Loss: 2.0458
Epoch [1/5], Step [300/600], Loss: 1.9931
Epoch [1/5], Step [400/600], Loss: 1.9159
Epoch [1/5], Step [500/600], Loss: 1.8161
Epoch [1/5], Step [600/600], Loss: 1.7303
Epoch [2/5], Step [100/600], Loss: 1.6953
Epoch [2/5], Step [200/600], Loss: 1.7030
Epoch [2/5], Step [300/600], Loss: 1.5779
Epoch [2/5], Step [400/600], Loss: 1.5470
Epoch [2/5], Step [500/600], Loss: 1.5059
Epoch [2/5], Step [600/600], Loss: 1.5085
Epoch [3/5], Step [100/600], Loss: 1.4243
Epoch [3/5], Step [200/600], Loss: 1.2968
Epoch [3/5], Step [300/600], Loss: 1.3385
Epoch [3/5], Step [400/600], Loss: 1.3625
Epoch [3/5], Step [500/600], Loss: 1.2788
Epoch [3/5], Step [600/600], Loss: 1.2409
Epoch [4/5], Step [100/600], Loss: 1.2629
Epoch [4/5], Step [200/600], Loss: 1.2518
Epoch [4/5], Step [300/600], Loss: 1.2384
Epoch [4/5], Step [400/600], Loss: 1.0889
Epoch [4/5], Step [500/600], Loss: 1.1904
Epoch [4/5], Step [600/600], Loss:

In [7]:
# Test the model
"""
torch.no_grad:
Context-manager that disabled gradient calculation
In test phase, we don't need to compute gradients (for memory efficiency)
"""
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1) #max_value, max_index
        total += labels.size(0) #get 0-th axis's size
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the model on the 10000 test images: 83 %
